### *Run these cells only when in Google Colab*

In [ ]:
# Clone the repository
!git clone https://github.com/beerger/mad_seminar_ws23.git
# Move all content to the current directory
!mv ./mad_seminar_ws23/* ./
# Remove the empty directory
!rm -rf mad_seminar_ws23/

In [ ]:
# Install additional packages
!pip install pytorch_lightning --quiet
!pip install lpips

## Imports for inference

In [ ]:
import pytorch_lightning as pl
import yaml
import torch
import matplotlib.pyplot as plt
import json
from google.colab import drive

from model.local_net import LocalNet
from model.dad import DADHead
from model.iad import iad_head
from model.global_net import GlobalNet

# autoreload imported modules
%load_ext autoreload
%autoreload 2

## Load and visualize data

Mount current Colab session to Google Drive (training/val images are stored here)

In [ ]:
# Will provide you with an authentication link
drive.mount('/content/drive')

Copy zipped file of zipper dataset from Google Drive to current Colab session

In [ ]:
!cp "/content/drive/MyDrive/AnomalyDetection/Datasets/MVTec/zipper.tar.xz" "/content/"
# Unzip it
!tar -xf /content/zipper.tar.xz -C /content/
# Remove zip file
!rm -rf zipper.tar.xz

Get paths to test images

In [ ]:
import os

# Specify the directory you want to list
train_directory = '/content/zipper/test/'

# Get a list of all files in the train_directory
file_list = [train_directory + f for f in os.listdir(train_directory) if os.path.isfile(os.path.join(train_directory, f))]

print(file_list)